# 100k DOFs case

From the Kratos outputs it seems that on every "Fluid Dynamics Analysis" step 6
problems are solved using AMGCL: 2 larger ones, and 4 smaller ones (a larger
problem has exactly 4 times more DOFs than a smaller one). Riccardo says these
are all physically different problems, which seems to be confirmed by the fact
that each of the problems takes distinctive and repeated across time steps
number of iterations to converge. I'll consider each of these as a separate
sequence of problems.

AMGCL parameters:

- `solver.type=bicgstab`
- `solver.tol=1e-6`
- `precond.coarse_enough=500`
- `precond.coarsening.type=aggregation`
- `precond.relax.type=damped_jacobi`

The larger problems were solved with 4x4 block values both with OpenMP and VexCL(CUDA) backends.

In [1]:
import pandas
import re
from pathlib import Path
def csv(fname):
    return pandas.read_csv(fname, delimiter='\t', header=None)

def speedup(full, part):
    data = {}
    for fname in (full, part):
        for line in open(fname):
            if m := re.search(r'amgcl:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'total')] = float(m.group(1))
            elif m := re.search(r'setup:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'setup')] = float(m.group(1))
    print(f"Total speedup: {100 * (data.get((full, 'total'), 0) / data.get((part, 'total'), 1) - 1):.2f}%")
    print(f"Setup speedup: {100 * (data.get((full, 'setup'), 0) / data.get((part, 'setup'), 1) - 1):.2f}%")
    
def report(size, device, num):
    full = Path(f'{size}/{device}-{num}-full.time')
    part = Path(f'{size}/{device}-{num}-part.time')
    
    print('*** Full AMGCL rebuild on every step ***\n')
    print(full.read_text().strip())
    print(f"\nAverage iterations: {csv(full.with_suffix('.log')).mean()[2]:.1f}")
    
    print('\n\n*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***\n')
    print(part.read_text().strip())
    print(f"\nAverage iterations: {csv(part.with_suffix('.log')).mean()[2]:.1f}")
    speedup(full, part)

## Small problem 0, OpenMP

In [2]:
report('small', 'cpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:       16.561 s] (100.00%)
[ self:          0.043 s] (  0.26%)
[  amgcl:        4.225 s] ( 25.51%)
[    setup:      1.354 s] (  8.18%)
[      full:     1.354 s] (  8.18%)
[    solve:      2.871 s] ( 17.33%)
[  reading:     12.293 s] ( 74.23%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           3.782 s] (100.00%)
[ self:             0.013 s] (  0.33%)
[  amgcl:           3.486 s] ( 92.17%)
[    setup:         0.721 s] ( 19.06%)
[      full:        0.123 s] (  3.26%)
[      partial:     0.598 s] ( 15.81%)
[    solve:         2.765 s] ( 73.11%)
[  reading:         0.283 s] (  7.49%)

Average iterations: 5.0
Total speedup: 21.20%
Setup speedup: 87.79%


## Small problem 0, VexCL(CUDA)

In [3]:
report('small', 'gpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:        3.122 s] (100.00%)
[ self:          0.010 s] (  0.33%)
[  amgcl:        2.844 s] ( 91.09%)
[    setup:      2.033 s] ( 65.12%)
[      full:     2.033 s] ( 65.12%)
[    solve:      0.811 s] ( 25.97%)
[  reading:      0.268 s] (  8.58%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           2.353 s] (100.00%)
[ self:             0.008 s] (  0.33%)
[  amgcl:           2.083 s] ( 88.49%)
[    setup:         1.311 s] ( 55.70%)
[      full:        0.193 s] (  8.22%)
[      partial:     1.117 s] ( 47.48%)
[    solve:         0.772 s] ( 32.79%)
[  reading:         0.263 s] ( 11.18%)

Average iterations: 5.0
Total speedup: 36.53%
Setup speedup: 55.07%


## Small problem 2, OpenMP

In [4]:
report('small', 'cpu', 2)

*** Full AMGCL rebuild on every step ***

[Profile:       26.907 s] (100.00%)
[ self:          0.066 s] (  0.25%)
[  amgcl:       10.741 s] ( 39.92%)
[    setup:      1.492 s] (  5.54%)
[      full:     1.492 s] (  5.54%)
[    solve:      9.249 s] ( 34.37%)
[  reading:     16.100 s] ( 59.84%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:          10.343 s] (100.00%)
[ self:             0.062 s] (  0.60%)
[  amgcl:           9.992 s] ( 96.60%)
[    setup:         0.746 s] (  7.21%)
[      full:        0.129 s] (  1.24%)
[      partial:     0.617 s] (  5.97%)
[    solve:         9.246 s] ( 89.39%)
[  reading:         0.290 s] (  2.80%)

Average iterations: 15.6
Total speedup: 7.50%
Setup speedup: 100.00%


## Small problem 2, VexCL(CUDA)

In [5]:
report('small', 'gpu', 2)

*** Full AMGCL rebuild on every step ***

[Profile:        4.799 s] (100.00%)
[ self:          0.008 s] (  0.16%)
[  amgcl:        4.523 s] ( 94.25%)
[    setup:      1.889 s] ( 39.37%)
[      full:     1.889 s] ( 39.37%)
[    solve:      2.634 s] ( 54.88%)
[  reading:      0.268 s] (  5.58%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           4.180 s] (100.00%)
[ self:             0.008 s] (  0.18%)
[  amgcl:           3.912 s] ( 93.57%)
[    setup:         1.316 s] ( 31.49%)
[      full:        0.196 s] (  4.70%)
[      partial:     1.120 s] ( 26.79%)
[    solve:         2.595 s] ( 62.08%)
[  reading:         0.261 s] (  6.24%)

Average iterations: 15.6
Total speedup: 15.62%
Setup speedup: 43.54%


## Large problem 0, OpenMP

In [6]:
report('large', 'cpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:      310.242 s] (100.00%)
[  amgcl:       77.673 s] ( 25.04%)
[    setup:      7.109 s] (  2.29%)
[      full:     7.109 s] (  2.29%)
[    solve:     70.564 s] ( 22.74%)
[  reading:    232.466 s] ( 74.93%)

Average iterations: 16.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         301.478 s] (100.00%)
[  amgcl:          77.133 s] ( 25.58%)
[    setup:         5.859 s] (  1.94%)
[      full:        0.750 s] (  0.25%)
[      partial:     5.109 s] (  1.69%)
[    solve:        71.274 s] ( 23.64%)
[  reading:       224.252 s] ( 74.38%)

Average iterations: 16.9
Total speedup: 0.70%
Setup speedup: 21.33%


## Large problem 0, VexCL(CUDA)

In [7]:
report('large', 'gpu', 0)

*** Full AMGCL rebuild on every step ***

[Profile:      265.118 s] (100.00%)
[  amgcl:       34.535 s] ( 13.03%)
[    setup:     13.223 s] (  4.99%)
[      full:    13.223 s] (  4.99%)
[    solve:     21.311 s] (  8.04%)
[  reading:    230.508 s] ( 86.95%)

Average iterations: 16.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         252.079 s] (100.00%)
[  amgcl:          32.506 s] ( 12.90%)
[    setup:        10.958 s] (  4.35%)
[      full:        1.363 s] (  0.54%)
[      partial:     9.594 s] (  3.81%)
[    solve:        21.548 s] (  8.55%)
[  reading:       219.497 s] ( 87.07%)

Average iterations: 16.9
Total speedup: 6.24%
Setup speedup: 20.67%
